# Auxiliary

In [9]:
import numpy as np
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Models